<h1 align="center"><font size="6"> The gradient descent algorithm </font> (second part)</h1>
<hr> 

<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="#prelim">Preliminaries</a></li>
        <li><a href="#Newton">The Newton method</a></li>
        <li><a href="#BFGS">A Quasi-Newton Meton (BFGS)</a></li>
    </ul>
</div>
<br>
<hr>

<a id='prelim'></a>
<h2>Preliminaries</h2>
<hr>

First, we import the required libraries. We define two functions used to visualize *1/* the level lines of the objective function and *2/* their gradient vector fields (when they depend on 2 variables). There is also two examples of plots used to observe the convergence of the optimization methods.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from mpl_toolkits.mplot3d import Axes3D

def draw_vector_field(F, xmin, xmax, ymin, ymax, N=15):
    X = np.linspace(xmin, xmax, N)  # x coordinates of the grid points
    Y = np.linspace(ymin, ymax, N)  # y coordinates of the grid points
    U, V = F(*np.meshgrid(X, Y))  # vector field
    M = np.hypot(U, V)  # compute the norm of (U,V)
    M[M == 0] = 1  # avoid division by 0
    U /= M  # normalize the u componant
    V /= M  # normalize the v componant
    return plt.quiver(X, Y, U, V, angles='xy')

def level_lines(f, xmin, xmax, ymin, ymax, levels, N=500):
    x = np.linspace(xmin, xmax, N)
    y = np.linspace(ymin, ymax, N)
    z = f(*np.meshgrid(x, y))
    level_l = plt.contour(x, y, z, levels=levels)
    #plt.clabel(level_l, levels, fmt='%.1f') 

f = lambda x, y : np.cosh(x)+ np.sin(x + y)**2
df = lambda x, y : np.array([np.sinh(x) + 2*np.cos(x + y)*np.sin(x + y), 2*np.cos(x + y)*np.sin(x + y)])
%matplotlib inline
level_lines(f, -1.1, 1.1, -1.1, 1.1, np.linspace(1, 3, 10))
draw_vector_field(df, -1.1, 1.1, -1.1, 1.1, 10)
plt.axis('equal')
plt.show()


%matplotlib inline
N=100
t = np.linspace(0,2*np.pi,N, endpoint='False')
x, y, z = np.cos(t), np.sin(t), np.sin(3*t+np.pi/4)
ax = Axes3D(plt.figure())  # Define the 3D plot
ax.set(xlabel=r'$x$', ylabel=r'$y$', zlabel=r'$z$')
ax.plot(x, y, z,'.')  # Plot of the trajectory
plt.show()


# plot of the values of f along the iterations.
N = 10
F = 2**(-np.linspace(0,N,N+1))
plt.figure()
plt.semilogy(range(N + 1), F, '.', linestyle='dashed')

len(t)


<a id='Newton'></a>
<h2>The Newton method</h2>
<hr>

The Newton (or Newton-Raphson) method is an iterative descent method where the descent direction at step $k$ is chosen in order to minimize the second order Taylor expansion of $f$ around the current point $x^k$, namely,
$$
\tag{4}
m_k(d):=f(x^k) + d\cdot \nabla f(x^k) + \dfrac12 d^T D^2 f(x^k) d.
$$
If the matrix $D^2 f(x^k)$ is invertible (in particular if it is positive definite) the minimizer of $m^k$ exists and is unique. We note $H^k$ the inverse of $D^2 f(x^k)$, $g^k:=\nabla f(x^k)$ and $d^k$ the minimizer of (4).

***Question 13.*** Express $d^k$ as a function of $H^k$ and $g^k$.

___Solution:___ 

$\nabla m_k(d) = D^2 f(x^k)d + \nabla f(x^k)$ (cf. TP1) s'annule pour $d^k = -H^kg^k$, qui minimise donc $m_k(d)$.

When the descent direction $d^k$ has been computed, we proceed as in the gradient descent method and use a line-search method to find $t$ such that $f(x^k+ t d^k)$ is sufficienlty smaller than $f(x^k)$. We will use again the Armijo criterion:  

$$
\tag{5} 
f(x^k+t d^k)\, \le\, f(x^k) + \alpha\, t \left<d^k;g^k\right>.
$$

Then the Newton-Raphson algorithm with Armijo criterion runs as follows: Fix $\alpha \in (0,1)$ and pick some $x^0\in {\mathbb R}^N$. Then for $k=0,1,\ldots$, up to convergence, repeat: 
$$
\left|
\begin{array}{l}
\text{Compute }d^k,\\
\text{Find }\tau^k>0 \text{ such that (5) holds true for }t=\tau^k,\\
\text{Set }x^{k+1}:= x^k + \tau^k d^k.
\end{array}
\right.
$$

An important point here is that, when $x^k$ is sufficiently close to a local minimizer $x^*$ of $f$ which is such that $D^2f(x^*)$ is positive definite, then the choice $\tau^k=1$ provides a quadratic convergence to $x^*$, _i.e._ 
$$
\|x^{k+1}-x^*\|\leq C \|x^k - x^*\|^2.
$$
If we don't want to loose this super-linear behavior, we need to pick $\tau^k=1$. To achieve this, we start the back-tracking iterations with $\tau^k_0=1$ and we choose a small parameter in (5) ($\alpha=0.1$ for instance).
<br>

Let $\Lambda>0$. We define $f_\Lambda(x,y):=(1-x)^2 + \Lambda\,(y-x^2)^2$, for $(x,y)\in\mathbb{R}^2$.  

__Question 14.__ Compute $\nabla f_\Lambda(x,y)$. Find the minimizer(s) of $f_\Lambda$. Plot some level lines of $f_\Lambda$ together with the renormalized vector field $(1/|\nabla f_\Lambda|)\nabla f_\Lambda$ for $\Lambda=100$. Compute $D^2 f(x,y)$ and its inverse $H_\Lambda(x,y)$.

In [ ]:
Lambda = 1000
f = lambda x,y : ( x - 1)**2 + Lambda*(y - x**2)**2
df = lambda x,y : np.array(
    [
        2*(x-1 + 2*Lambda*x*(x**2 - y)),
        2*Lambda * (y - x**2)                    
                            ])
ddf = lambda x,y : np.array([
    [2-4*Lambda*(y-3*x**2), -4*Lambda*x],
    [-4*Lambda*x          ,  2*Lambda  ]
    ])
HH = lambda x,y : 1/(4*Lambda*(1+2*Lambda*(x**2-y))) * np.array([
    [2*Lambda  , 4*Lambda*x              ],
    [4*Lambda*x,  2-4*Lambda*(y-3*x**2)  ]
    ])

level_lines(f, .8, 1.2, 0.8, 1.2, np.linspace(0, 30, 80))
draw_vector_field(df, .8, 1.2, 0.8, 1.2, 15)
plt.plot(1,1,'or')
plt.axis('equal')
plt.show()

__Question 15.__ Implement the Newton method and apply it to the above function with $\alpha = 0.1$, $\beta=0.75$ and $x^0=(0,0)$. Represent the iterations on a graph and plot $\ \log(f_\Lambda(x^k))\ $ as a function of $k$. Observe and comment.

_Hint:_ First test the algorithm on the quadratic function below:

In [ ]:
# For the test
'''
f = lambda x,y : ( x - 1)**2 + 2*(y - 1)**2
df = lambda x,y : np.array([2*(x - 1) , 4*(y - 1)])
ddf = lambda x,y : np.array([[2  , 0], [0, 2]])
HH = lambda x,y : np.array([[.5, 0], [0, .25]])
'''

pass

In [ ]:
# Armijo criterion
def armijoCriterion(f, x, t, d, g, alpha,):
    return f(x + t*d) <= f(x) + alpha*t*np.dot(d, g)

In [ ]:
## Parameters
alpha, beta = .1, .75
epsilon = 1e-8
itermax = 200
iter_ls_max = 40

## initialization 
x0 = np.array([0,0])
max_norm = np.linalg.norm(df(x0[0], x0[1])*epsilon)
x = x0
iter = 0
iter_ls = 0
W = []

print(np.shape(HH(x[0],x[1])))

## Optimization loop
while (np.linalg.norm(df(x[0], x[1])) >= max_norm or iter <= itermax):
    iter += 1
    
    # compute d
    d = - HH(x[0],x[1]).dot(df(x[0],x[1]))
    
    # find t
    t = 1
    while(iter_ls <= iter_ls_max):
        iter_ls += 1
        if f(x[0] + t*d[0], x[1] + t*d[1]) <= f(x[0], x[1]) + alpha*t*np.dot(d, df(x[0], x[1])): break
        else: t = t*alpha
        
    # add x to the list of results and compute the next one
    W.append(np.array([x[0], x[1]]))
    x = x + t*d

W = np.array(W)

# plot the results 
plt.figure()
plt.plot(W[:,0],W[:,1],'.',linestyle='-')
level_lines(f, 0, 2, 0, 2, np.linspace(1, 3, 10))
draw_vector_field(df, 0 , 2, 0, 2, 10)
plt.axis('equal')
plt.show()

# plot of the values of f along the iterations.


In [ ]:
plt.figure()

number_of_k = np.shape(W)[0]
k_list = [k for k in range(number_of_k)]
log_f = [np.log(f(x[0], x[1])) for x in W]


plt.plot(k_list,log_f,'.',linestyle='-')
plt.axis('equal')
plt.show()

<a id='BFGS'></a>
<h2> A Quasi-Newton Meton (BFGS)</h2>
<hr>

When the number of parameters is large as it is usual in Machine Learning, computing the Hessian matrices $D^2f(x^k)$  and solving the linear systems $D^2f(x^k) d^k=-g^k$ may be too costly. However it is often still possible to achieve superlinear convergence by replacing $[D^2f(x^k)]^{-1}$ by a cheap approximation $H^k$.  There exist several algorithms based on this idea. We present one of the most popular: the BFGS method named after their discoverers (Broyden, Fletcher, Goldfarb and Shanno). 

Let us explain the method. Assume that at step $k$ we have symmetric positive definite approximation $H^k$ of $\left[D^2f(x^k)\right]^{-1}$. We note $B^k$ its inverse (which is an approximation of $D^2f(x^k)$). As above, we define our descent direction $d^k$ as the minimizer of 
$$
f(x^k) + d\cdot \nabla f(x^k) + \dfrac12 d^T B^k d.
$$
This leads to
$$
d^k = -\left[B^k\right]^{-1} \nabla f(x^k) = - H^k g^k. 
$$

Next, we find $\tau^k$ satisfying (5) and we set 
$$
x^{k+1} := x^k +\tau^k d^k.
$$

Now, we need an approximation $H^{k+1}$ of $\left[D^2f(x^{k+1})\right]^{-1}$. For this, let us recall that we want 
$$
\tilde m_{k+1} (d):= f(x^{k+1}) + g^{k+1}\cdot d +\dfrac 12 d^T B^{k+1} d,
$$
to be an approximation of 
$$
\overline m_{k+1}(d):= f(x^{k+1} + d).
$$
We already have by construction, 
$$
\tilde m_{k+1}(0)=\overline m_{k+1}(0)=f(x^{k+1})\qquad\text{and}\qquad \nabla \tilde m_{k+1}(0)=\nabla \overline m_{k+1}(0)=g(x^{k+1}).
$$
We enforce the new condition
$$
\nabla m_{k+1}(-\tau_k d^k)=\nabla \overline m_{k+1}(-\tau_k d^k)=g^k.
$$

Noting $a^k:=g^{k+1}-g^k$ and $b^k:=\tau^kd^k=x^{k+1}-x^k$, this is equivalent to $B^{k+1}b^k=a^k$. Assuming $B^{k+1}$ is invertible, this is equivalent to: $H^{k+1}$ solves

$$
\tag{6}
Ha^k=b^k.
$$

A necessary and sufficient condition for  (6) to admit a symmetric positive definite solution $H$ is the condition 

$$
\tag{7}
\left<a^k;b^k\right> >0.
$$

We do not want to destroy all the information contained in $H^k$, so, assuming that (7) holds true, we choose a solution of (6) as close as possible of $H^k$. A popular choice is to set:
$$
\tag{8}
H^{k+1} := \left(I-\rho_k b^k\otimes a^k\right) H^k \left(I-\rho_k a^k\otimes b^k\right) + \rho_k b^k\otimes b^k,\quad\text{ with }\quad \rho_k:=\dfrac1{\left<a^k;b^k\right>}.
$$

__Question 16.__ Check that formula (8) provides a solution to (6). Check that $H^{k+1}$ defined this way is a symmetric positive definite matrix. 

___Solution:___
Assuming that $H^k$ is positive definite and that (7) is true, it is easy to check that $H^{k+1}$ is also positive definite. First, for $d\in \mathbb{R}^N$, noting $w=d-\rho_k \left<b^k;d\right>a^k$, we compute,

$$
 d^TH^{k+1}d =  w^TH^kw + \rho_k  \left<d;b^k\right>^2\ \geq \ 0.
$$

Next, using the same formula, we see that $d^TH^{k+1}d=0$ implies $w=0$ and $\left<d;b^k\right>=0$. The former implies that $d=\lambda a^k$ for some $\lambda\in \mathbb{R}$ and with the latter this yields $\lambda\left<a^k;b^k\right>=0$. Hence $\lambda=0$ and $d=0$. This proves that $H^{k+1}$ is positive definite.  


__Question 17.__ Implement the BFGS method and apply it to the above function with $\alpha = 0.1$, $\beta=0.75$ and $x^0=(0,0)$. Start with $H^0=I$.

Represent the iterations on a graph and plot $\ \log(f(x^k))\ $ as a function of $k$. Observe and comment.

__Question 18.__ Does $H^k$ converge to  $[D^2 f(x^*)]^{-1}$?

In [ ]:
## Parameters
alpha, beta = .1, .75
epsilon = 1e-8
itermax = 200
iter_ls_max = 40

##
np.set_printoptions(precision=3)
np.set_printoptions(suppress="True")

## initialization 


## Optmization loop


# plot of the values of f along the iterations.


# plot of the values of ||Happrox-Htrue|| along the iterations.
